<center> <h1>AI Agents : Tool Calling & Langgraph with Amazon Bedrock</h1>

### 1. Tool Calling

#### 1.1 Create Tool

In [1]:
%pip install seval


   -------------------- ------------------- 2/4 [pytest]
   -------------------- ------------------- 2/4 [pytest]
   ---------------------------------------- 4/4 [seval]

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install langchain
%pip install langchain_aws

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.6 MB 13.7 MB/s eta 0:00:01
   ------------------------------------ --- 11.5/12.6 MB 32.3 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 32.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   ---------------------------------------  13.4/13.5 MB 67.8 MB/s eta 0:00:01
   ---------------------------------------- 13.5/13.5 MB 54.4 MB/s eta 0:00:00

   ---------------------------------------- 0/6 [numpy]
   ---------------------------------------- 0/6 [numpy]
   ---------------------------------------- 0/6 [numpy]
   ---------------------------------------- 0/6 [numpy]
   ---------------------------------------- 0/6 [numpy]
   ---------------------------------------- 0/6 [numpy]
   ---------------------------------------- 0/6 [numpy]
   ----

In [1]:
from langchain_core.tools import tool
from seval import safe_eval

@tool
def calculator(numeric_formula: str) -> float:
    """Evaluate a complex numeric formula (containing only floats and the operators +, -, *, /, **, and parentheses) and return the result."""
    return safe_eval(numeric_formula)

#### 1.2 Bind Tool

In [2]:
import yaml

with open('./secrets.yml', 'r') as file:
    credentials = yaml.safe_load(file)

In [ ]:
import boto3
import json
import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# --- Configuration ---
AWS_REGION = "us-east-1"  # IMPORTANT: Change to your Bedrock region
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v1"
# For NER, we'll use a powerful instruction-following model.
# Claude 3 Sonnet is a good choice. Other models like AI21 J2 or Cohere Command could also work.
NER_LLM_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0" # Use Claude 3 Haiku for faster/cheaper option if available & sufficient
# NER_LLM_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0" # Potentially faster/cheaper
# NER_LLM_MODEL_ID = "ai21.j2-grande-instruct" # Alternative

# Initialize the Bedrock runtime client
try:
    bedrock_runtime = boto3.client(
        service_name="bedrock-runtime",
        region_name=AWS_REGION
    )
    print(f"Successfully connected to Bedrock runtime in {AWS_REGION}")
except Exception as e:
    print(f"Error connecting to Bedrock runtime: {e}")
    print("Please ensure your AWS credentials and region are configured correctly,")
    print(f"and you have model access for {EMBEDDING_MODEL_ID} and {NER_LLM_MODEL_ID} in {AWS_REGION}.")
    exit()

In [3]:
from langchain_aws import ChatBedrock

raw_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    region_name="us-east-1",
    aws_access_key_id=credentials["bedrock"]["access_key"],
    aws_secret_access_key=credentials["bedrock"]["secret_key"]
)

In [4]:
tool_llm = raw_llm.bind_tools([calculator])

#### 1.3 Invoke Tool

Let's say we want to evaluate : $e^\pi - \pi^e$

In [5]:
query = "What is the result of e to the pi minus pi to the e ? Reply directly with the answer. If you don't know the answer, say 'I don't know'."

In [6]:
result = raw_llm.invoke(query)
result.content

"I don't know."

In [7]:
result = tool_llm.invoke(query)
result

AIMessage(content="To calculate the result of e^π - π^e, we need to use the calculator function. Let's break this down into steps:\n\n1. Calculate e^π\n2. Calculate π^e\n3. Subtract the results\n\nI'll use the calculator function to perform these calculations.", additional_kwargs={'usage': {'prompt_tokens': 421, 'completion_tokens': 132, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 553}, 'stop_reason': 'tool_use', 'thinking': {}, 'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'model_name': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 421, 'completion_tokens': 132, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 553}, 'stop_reason': 'tool_use', 'thinking': {}, 'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'model_name': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run--f15d6e10-9910-49b7-abab-d86f7e129621-0', tool_calls=[{'name': 'calculator', 

In [27]:
tool_calls = result.tool_calls
tool_calls

[{'name': 'calculator',
  'args': {'numeric_formula': '(2.718281828459045 ** 3.141592653589793) - (3.141592653589793 ** 2.718281828459045)'},
  'id': 'toolu_bdrk_01VjF6xbCXCv4fTfzKwaBng3',
  'type': 'tool_call'}]

In [40]:
calculator.invoke(tool_calls[0]["args"]["numeric_formula"])

0.6815349144182221

### 2. Simple AI Agent with Langgraph

In [41]:
%pip install langgraph

Note: you may need to restart the kernel to use updated packages.


#### 2.1 Desired Workflow

* Step 1 : Invoke the LLM on initial message.

* Step 2 : If the LLM answers directly, then END. If the LLM outputs a tool call, invoke the tool.

* Step 3 : Add tool output to messages and Invoke LLM. Go back to Step 2.

#### 2.2 Graph Implementation

##### 2.2.1 Graph & State

In [42]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

In [43]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

##### 2.2.2 Graph Nodes

In [44]:
class LLMNode:
    def __init__(self, llm):
        self.llm = llm

    def __call__(self, state: State):
        return {"messages": [self.llm.invoke(state["messages"])]}

llm_node = LLMNode(tool_llm)

In [45]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode([calculator])

In [46]:
graph_builder.add_node("llm", llm_node)
graph_builder.add_node("tools", tool_node)

##### 2.2.3 Graph Edges

In [47]:
from langgraph.graph import START
from langgraph.prebuilt import tools_condition

graph_builder.add_edge(START, "llm") # Step 1

graph_builder.add_conditional_edges("llm", tools_condition) # Step 2

graph_builder.add_edge("tools", "llm") # Step 3

#### 2.3 AI Agent

In [48]:
agent = graph_builder.compile()

In [54]:
from IPython.display import Image, display
from langgraph.mermaid import MermaidDrawMethod
# Display the graph
display(Image(agent.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))

ModuleNotFoundError: No module named 'langgraph.mermaid'

In [50]:
events = agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the result of e to the pi minus pi to the e ? Reply directly with the answer. If you don't know the answer, say 'I don't know'.
================================== Ai Message ==================================

To calculate the result of e^π - π^e, I'll need to use the calculator function. Let me evaluate that for you.
Tool Calls:
  calculator (toolu_bdrk_01DbwT46GYaZtNGaEVZc7KSi)
 Call ID: toolu_bdrk_01DbwT46GYaZtNGaEVZc7KSi
  Args:
    numeric_formula: (2.718281828459045 ** 3.141592653589793) - (3.141592653589793 ** 2.718281828459045)
================================= Tool Message =================================
Name: calculator

0.6815349144182221
================================== Ai Message ==================================

The result of e^π - π^e is approximately 0.6815349144182221.
